# 큐펭 크롤러
* Wanted에서 데이터 분석가 공고 크롤링하기

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [1]:
import pandas as pd
from selenium import webdriver
from bs4 import BeautifulSoup
import re
from tqdm import tqdm_notebook
import time

In [14]:
driver = webdriver.Chrome('chromedriver/chromedriver.exe')

In [15]:
# 함수 지정

# 주요업무, 자격요건, 우대사항 가져오기
def FindElements(keyword, b):
    b = str(b)
    start = b.find('<h6>{}</h6>'.format(keyword))+22    # +22: header태그와 p, span 태그 제거
    end = b[start:].find('</p>')-7    # span태그 제거
    str_ = b[start:start+end].replace('<br/>', '\n')
    return str_

# 스크롤 끝까지 내리기
def ScrollToBottom(t=0.5):
    # Get scroll height
    last_height = driver.execute_script("return document.body.scrollHeight")
    while True:
        # Scroll down to bottom
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

        # Wait to load page
        time.sleep(t)

        # Calculate new scroll height and compare with last scroll height
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

In [16]:
# 사람인 페이지 접속
driver.get('https://www.wanted.co.kr/wdlist/507/656?country=kr&\
            job_sort=job.latest_order&years=-1&locations=all')
ScrollToBottom()
driver.implicitly_wait(10)
driver.execute_script("window.scrollTo(0, 0);")

In [17]:
# dict 선언
jd = {'name': [], 'position': [], 'wanted_cat': [], 'about': [], 'roll': [], 'qual': [],
      'pref': [], 'category': [], 'tags': [], 'link': []}

In [18]:
dict_url = {'머신러닝 엔지니어': 'https://www.wanted.co.kr/wdlist/518/1634?country=kr&job_sort=job.latest_order&years=-1&locations=all',
            '데이터 엔지니어': 'https://www.wanted.co.kr/wdlist/518/655?country=kr&job_sort=job.latest_order&years=-1&locations=all',
            '데이터 사이언티스트': 'https://www.wanted.co.kr/wdlist/518/1024?country=kr&job_sort=job.latest_order&years=-1&locations=all',
            '데이터 분석가': 'https://www.wanted.co.kr/wdlist/507/656?country=kr&job_sort=job.latest_order&years=-1&locations=all'}

In [19]:
for wanted_cat, url in zip(dict_url.keys(), dict_url.values()):
    # 사람인 페이지 접속
    driver.get(url)
    for _ in range(10):
        ScrollToBottom()
        driver.implicitly_wait(10)
        time.sleep(1)
        
    driver.execute_script("window.scrollTo(0, 0);")
    
    # 포지션 수 찾기
    soup = driver.page_source
    b = BeautifulSoup(soup, 'lxml')
    count_position = len(str(b.find_all('ul', 'clearfix')).split('<li>'))-1

    driver.implicitly_wait(10)
    time.sleep(2)

    # 크롤링 시작
    for num in tqdm_notebook(range(1, count_position+1)):
        # 스크롤 내리기
        try:
            driver.find_element_by_css_selector('#__next > div > div._1yHloYOs_bDD0E-s121Oaa > \
            div._2y4sIVmvSrf6Iy63okz9Qh > div > ul > li:nth-child({}) > div > a > header'.format(num)).click()
        except:
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")# 공고 클릭

        # 페이지 로딩 대기
        driver.implicitly_wait(10)
        time.sleep(3)
        # 페이지 소스 가져오기
        soup = driver.page_source
        b = BeautifulSoup(soup, 'lxml')
        time.sleep(1)
        # -------------------------------------------
        # ----------------크롤링 시작----------------

        # 회사소개
        jd['about'].append(driver.find_element_by_css_selector('#__next > div > div._37L2cip40tqu3zm3KC4dAa > div._17tolBMfrAeoPmo6I9pA1P > div._1FVm15xN253istI2zLF_Ax > div > div._31EtVNPZ-KwYCXvVZ3927g > section._1LnfhLPc7hiSZaaXxRv11H > p:nth-child(1) > span').text)
        # 주요업무, 자격요건, 우대사항
        jd['roll'].append(FindElements('주요업무', b))
        jd['qual'].append(FindElements('자격요건', b))
        jd['pref'].append(FindElements('우대사항', b))

        # 회사명, 회사 카테고리, 직무명, 링크, 원티드 카테고리
        jd['name'].append(b.find_all('h6')[0].text)
        jd['category'].append(b.find_all('h6')[-1].text)
        jd['position'].append(b.find_all('h2')[0].text)
        jd['link'].append(driver.current_url)
        jd['wanted_cat'].append(wanted_cat)

        # 태그
        p = re.compile('#{1}[\w,\s]*')
        hashtags = str(b.find_all('div', 'ObubI7m2AFE5fxlR8Va9t', 'li'))    # 해시태그 찾아내기
        jd['tags'].append(list(map(lambda x: x[1:], p.findall(hashtags))))    # 해시태그에서 '#' 지우기
        # ----------------크롤링 끝----------------
        # -----------------------------------------

        # 매크로 방지
        time.sleep(1)

        # 뒤로 가기
        driver.back()
        driver.implicitly_wait(10)

C:\Users\samsung\Anaconda3\lib\site-packages\ipykernel_launcher.py:20: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


In [20]:
df = pd.DataFrame(jd)

In [24]:
df.to_csv('source/cute_penguins_1/wanted_raw.csv', index=False)

In [22]:
df.head(3)

,name,position,wanted_cat,about,roll,qual,pref,category,tags,link
0,룩인바디,[인바디 앱] Data Scientist,머신러닝 엔지니어,"인바디에서 2018년 분사 후, \n200만+ 사용자, 2,500만+ 인바디데이터의...",- 데이터 파이프라인 중 개선이 필요한 사항이 무엇인지 검토하고 개선안을 도출하여 ...,- 데이터 관련 업무를 1년 이상 수행하신 분\n- 주도적인 문제 해결의 경험이 있...,"- 룩인바디와 함께 성장하길 희망하며, 다양한 역할 수행에 대해 열려있는 분\n- ...","IT, 컨텐츠","[연봉업계평균이상, 인원급성장, 50명이하, 설립3년이하, 성과급, 생일선물, 결혼...",https://www.wanted.co.kr/wd/34549
1,투블럭에이아이,AI 자연어처리 관련 서비스 개발,머신러닝 엔지니어,최신 Deep NLP 기술의 현실 업무 적용의 경험을 통해서 인공지능과 업무자동화의...,"NLP 관련 기술개발 \nNER , MRC 등 Task 모듈 개발 \n데이터 정제 ...",• NP 관련 기술 Stack 보유자 (단순히 자연어처리에 관심이 있으신 분은 거절...,"• 전산학 석사 이상, 자연어처리 전공자 우대, 검색 포탈 경력자 우대\n• 유관 ...","IT, 컨텐츠","[인원급성장, 50명이하, 설립3년이하, 인공지능, IT, 컨텐츠]",https://www.wanted.co.kr/wd/44279
2,뉴로핏(NEUROPHET),의료영상 인공지능 엔지니어,머신러닝 엔지니어,"뉴로핏은 뛰어난 뇌공학, 의료인공지능 기술을 실제 의료현장에 적용하여 의료진의 불편...","• 의료영상 기반 분석방법론(신경망 포함) 개발, 검증, 업데이트\n• 의료영상을 ...",• 신경망관련 연구 2년 이상\n• Tensorflow 사용 능력 (중급)\n• P...,• 의료 인공지능 구현 경험자 (segmentation/classification)...,"전문, 과학기술","[연봉상위20, 인원급성장, 50명이하, 설립3년이하, 스타트업, 커피, 재택근무,...",https://www.wanted.co.kr/wd/18473


In [23]:
df.tail(3)

,name,position,wanted_cat,about,roll,qual,pref,category,tags,link
536,윙블링,"데이터분석, 그로스해킹, PM",데이터 분석가,윙블링 소개 \n\n윙블링은 '아름다움에 날개를달다'라는 미션과 함께 빠르게 성장...,"• 데이터분석 및 시각화 (자사몰, Paid Marketing 채널, SCM 등)\...","• 커머스 분야 퍼포먼스 마케팅, 자사몰 데이터 분석, UX/UI 기획 경험\n• ...",• 기술적으로 부족한 카페24 기능을 바탕으로 데이터를 고도화하여 분석해보신 분\n...,"판매, 유통","[간식, 50명이하, 설립10년미만, 퇴사율5, 스타트업, 커피, 판매, 유통]",https://www.wanted.co.kr/wd/34773
537,여기어때컴퍼니,VC전략팀 시니어,데이터 분석가,좋은 숙소에서 행복한 시간을!\n\n위드이노베이션은 숙박 여정의 불편함을 해결하고 ...,- 전사 전략 수립 및 핵심 지표 관리\n- 전사 및 사업부 단위 프로젝트(Valu...,- 경영/전략 컨설팅 경력 5~8년 이상 또는 플랫폼 사업 전략기획 경력 5~8년 ...,- 국내외 메이저 경영 및 전략 컨설팅펌 근무 경력\n- IT/커머스/O2O 전략/...,"IT, 컨텐츠","[연봉상위20, 1,000명이하, 설립10년미만, 퇴사율5, 누적투자100억이상, ...",https://www.wanted.co.kr/wd/30399
538,루나소프트,플랫폼 기획자,데이터 분석가,루나소프트는 카카오와 네이버 플랫폼을 활용한 챗봇 구축과 독자 기술을 활용한 CTI...,• 플랫폼 서비스 기획\n• 시스템 설계 초안 디자인 및 문서화\n• UI 기획서 ...,• 프로젝트 관리 경험\n• 데이터 관점의 구조적 사고가 가능한 분\n• 논리적 사...,• Database Modeling 가능하신 분\n• 개발자에서 기획자로 전향 하고...,"IT, 컨텐츠","[연봉상위20, 인원급성장, 300명이하, 설립10년미만, 상여금, 조식제공, 식비...",https://www.wanted.co.kr/wd/19416


In [25]:
df.shape

(539, 10)